<a href="https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/get-started/try-apache-beam-yaml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
#@title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

# Try Apache Beam - Yaml

In this notebook, we set up your development environment and write a simple pipeline using Yaml API. We'll run it locally, using the [DirectRunner](https://beam.apache.org/documentation/runners/direct/). You can explore other runners with the [Beam Capatibility Matrix](https://beam.apache.org/documentation/runners/capability-matrix/).

To navigate through different sections, use the table of contents. From **View**  drop-down list, select **Table of contents**.

To run a code cell, you can click the **Run cell** button at the top left of the cell, or by select it and press **`Shift+Enter`**. Try modifying a code cell and re-running it to see what happens.

To learn more about Colab, see [Welcome to Colaboratory!](https://colab.sandbox.google.com/notebooks/welcome.ipynb).

# Setup

First, you need to set up your environment, which includes installing `apache-beam` and downloading a text file from Cloud Storage to your local file system. We are using this file to test your pipeline.

In [49]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

def save_to_file(content, file_name):
  with open(file_name, 'w') as f:
    f.write(content)

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
run('mkdir -p data')
run('gsutil cp gs://apache-beam-samples/SMSSpamCollection/SMSSpamCollection data/SMSSpamCollection.csv')

>> pip install --quiet apache-beam

>> mkdir -p data

>> gsutil cp gs://apache-beam-samples/SMSSpamCollection/SMSSpamCollection data/SMSSpamCollection.csv
Copying gs://apache-beam-samples/SMSSpamCollection/SMSSpamCollection...
/ [1 files][466.7 KiB/466.7 KiB]                                                
Operation completed over 1 objects/466.7 KiB.                                    



## Inspect the data
Let’s see how our data looks like.

In [4]:
run('head data/SMSSpamCollection.csv')
run('wc -l data/SMSSpamCollection.csv')

>> head data/SMSSpamCollection.csv
ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code

This dataset is a `csv` file with 5,574 rows and 2 columns recording the following attributes separated by a tab sign:
1. `Column 1`: The label (either `ham` or `spam`)
2. `Column 2`: The SMS as raw text (type `string`)

## First pipeline
We’ll start with creating a pipeline which loads the data, filters out valid messages leaving spam, and saves only valid lines to a file.


In [107]:
pipeline = '''
pipeline:
  - type: ReadFromText
    name: SmsData
    file_pattern: data/SMSSpamCollection.csv

  - type: PyMap
    name: SplitLine
    input: SmsData
    fn: 'lambda line: line.split("\\t")'

  - type: PyFilter
    name: KeepSpam
    input: SplitLine
    keep: 'lambda row: row[0] == "spam"'

  - type: WriteToText
    name: SaveToText
    input: KeepSpam
    file_path_prefix: "data/result-pipeline-01"
    file_name_suffix: ".txt"
'''
save_to_file(pipeline, 'pipeline-01.yaml')

In this example, each transformation contains the 'input' key, but if the pipeline is linear, such as ours, we can let the inputs be implicit by designating the pipeline as a `chain` type.


In [122]:
pipeline = '''
pipeline:
  type: chain
  transforms:
    - type: ReadFromText
      name: SmsData
      file_pattern: data/SMSSpamCollection.csv

    - type: PyMap
      name: SplitLine
      fn: 'lambda line: line.split("\\t")'

    - type: PyFilter
      name: KeepSpam
      keep: 'lambda row: row[0] == "spam"'

    - type: WriteToText
      name: SaveToText
      file_path_prefix: "data/result-pipeline-01"
      file_name_suffix: ".txt"
'''
save_to_file(pipeline, 'pipeline-01-chain.yaml')

To run the pipeline locally, using a DirectRunner, you need to run the yaml's main python script, passing the `pipeline-01-chain.yaml` (or `pipeline-01.yaml`) file as an input:

In [123]:
run('python -m apache_beam.yaml.main --pipeline_spec_file=pipeline-01-chain.yaml')

>> python -m apache_beam.yaml.main --pipeline_spec_file=pipeline-01-chain.yaml
Building pipeline...
INFO:apache_beam.yaml.yaml_transform:Expanding "SmsData" at line 5 
INFO:apache_beam.yaml.yaml_transform:Expanding "SplitLine" at line 9 
INFO:apache_beam.yaml.yaml_transform:Expanding "KeepSpam" at line 13 
INFO:apache_beam.yaml.yaml_transform:Expanding "SaveToText" at line 17 
Running pipeline...



Let's verify the results and see the content of the output file.

In [125]:
run('cat data/result-pipeline-01-00000-of-00001.txt')

>> cat data/result-pipeline-01-00000-of-00001.txt
['spam', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]
['spam', "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"]
['spam', 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.']
['spam', 'Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030']
['spam', 'SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info']
['spam', 'URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net

If everything went well, you should see only spam messages from our input dataset. Congratulations, onto the next example!


## Count words in spam messages, select top 10 popular words and write results to a file

Now we'd like to write a pipeline which counts words occurring in spam messages, selects the most popular ones and writes the result to a file.


In [118]:
pipeline = '''
pipeline:

  # Read input data. Each line from the csv file is a String.
  - type: ReadFromText
    name: SmsData
    file_pattern: data/SMSSpamCollection.csv

  # Split each line into an array, where the first element is message label (ham or spam) and the second is the content of the message.
  - type: PyMap
    name: SplitLine
    input: SmsData
    fn: 'lambda line: line.split("\\t")'

  # Select only the rows that contain spam messages, based on the label.
  - type: PyFilter
    name: SpamMessages
    input: SplitLine
    keep: 'lambda row: row[0] == "spam"'

  # Using a regex, we'll split the content of the message (one long string) into words (list of strings)
  - type: PyFlatMap
    name: FindWords
    input: SpamMessages
    fn: |
      import re
      lambda line: re.findall(r"[a-zA-Z]+", line[1])

  # Transforming each word to lower case and combining it with a '1'. Result of this step are pairs (word: 1).
  - type: PyMap
    name: PairLoweredWordsWith1
    input: FindWords
    fn: 'lambda word: (word.lower(), 1)'

  # Using SumPerKey transform, we'll calculate the occurence of each word.
  - type: SumPerKey
    name: GroupAndSum
    input: PairLoweredWordsWith1

  # Select 10 most popular words. Input format to this step is a tuple (word: count),
  # so we provide the count (row[1]) as the key to compare the numbers
  - type: TopNLargest
    name: Largest
    input: GroupAndSum
    n: 10
    key: 'lambda row: row[1]'

  # Save results to a text file.
  - type: WriteToText
    name: SaveToText
    input: Largest
    file_path_prefix: "data/result-pipeline-02"
    file_name_suffix: ".txt"
'''
save_to_file(pipeline, 'pipeline-02.yaml')

Let's run the pipeline:

In [119]:
run('python -m apache_beam.yaml.main --pipeline_spec_file=pipeline-02.yaml')

>> python -m apache_beam.yaml.main --pipeline_spec_file=pipeline-02.yaml
Building pipeline...
INFO:apache_beam.yaml.yaml_transform:Expanding "SmsData" at line 3 
INFO:apache_beam.yaml.yaml_transform:Expanding "SplitLine" at line 7 
INFO:apache_beam.yaml.yaml_transform:Expanding "SpamMessages" at line 12 
INFO:apache_beam.yaml.yaml_transform:Expanding "FindWords" at line 22 
INFO:apache_beam.yaml.yaml_transform:Expanding "PairLoweredWordsWith1" at line 29 
INFO:apache_beam.yaml.yaml_transform:Expanding "GroupAndSum" at line 35 
INFO:apache_beam.yaml.yaml_transform:Expanding "Largest" at line 39 
INFO:apache_beam.yaml.yaml_transform:Expanding "PrintLines" at line 45 
INFO:apache_beam.yaml.yaml_transform:Expanding "SaveToText" at line 51 
Running pipeline...
[('to', 691), ('a', 393), ('call', 370), ('you', 299), ('your', 264), ('free', 228), ('the', 206), ('for', 204), ('now', 203), ('or', 192)]



To view the output:

In [121]:
run('head -n 20 {}-00000-of-*'.format("data/result-pipeline-02"))

>> head -n 20 data/result-pipeline-02-00000-of-*
[('to', 691), ('a', 393), ('call', 370), ('you', 299), ('your', 264), ('free', 228), ('the', 206), ('for', 204), ('now', 203), ('or', 192)]



## Summary
Congratulations! You've just ran yaml pipelines in Apache Beam.
For all the available operations visit the documentation: # todo(yaml) add url
For a list of available transforms, visit # todo(yaml) add url